In [1]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from scipy.interpolate import interp1d

# === Input Parameters ===
T_well_C = 40  # Desired temperature in °C
concentration = 2  # g/L
molecular_weight = 8  # MDa
eta7_exp = 20  # Experimental viscosity at 7.3 s⁻¹ in cP

T_well = T_well_C + 273  # Convert to Kelvin
T_ref = 298  # Reference temperature (25°C)

# === Initial Viscosity (at T_ref) ===
eta_in = np.exp(-3.7188 + (578.919 / (-137.546 + T_ref)))

# === Estimate Intrinsic Viscosity ===
ceta = np.arange(0.1, 100.1, 0.1)
eta_0 = 1 + ceta + 0.582 * ceta**2.009 + 0.022 * ceta**4
parc = -0.08212 * ceta
n = 1 - (0.6187 - 0.5203 * np.exp(parc))
parc2 = (n - 1) / 2
lambda_d = 0.251 + 1.54 * molecular_weight * ceta / (concentration * T_ref)
lambda_ = lambda_d * (0.810 + 0.0230 * ceta**2.438)
parc1 = (lambda_ * 7.3)**2
eta7 = 1 + (eta_0 - 1) * (1 + parc1)**parc2

# === Interpolate to find experimental ceta
interp_func = interp1d(eta7, ceta, fill_value="extrapolate")
ceta_exp = float(interp_func(eta7_exp))

# === Carreau-Yasuda Coefficients at Reference Temperature ===
eta_0C = 1 + ceta_exp + 0.582 * ceta_exp**2.009 + 0.022 * ceta_exp**4
parcC = -0.08212 * ceta_exp
nC = 1 - (0.6187 - 0.5203 * np.exp(parcC))
parc2C = (nC - 1) / 2
lambda_dC = 0.251 + 1.54 * molecular_weight * ceta_exp / (concentration * T_ref)
lambdaC = lambda_dC * (0.810 + 0.0230 * ceta_exp**2.438)

# === Shear Range ===
shear = np.arange(0.01, 1000.1, 0.1)
etaC = eta_in + (eta_0C - eta_in) * (1 + (lambdaC * shear)**2)**parc2C

# === Temperature-Adjusted Model ===
a_eta, b_eta, cc_eta, d_eta = 0.000762728, 0.089195454, 0.004984355, 0.446385267
e_eta, f_eta, g_eta, h_eta = 0.005152396, 0.354050723, 0.004815493, 0.484329949
i_eta, j_eta, k_eta, l_eta = 0.018517312, 1.632479454, 0.0002, 0.029161594
m_eta, n_eta = 0.021472011, 3.475445058

a_lamb, b_lamb, c_lamb, d_lamb = 0.00189, 1, 0.0003, 0.02157
e_lamb, f_lamb, g_lamb, h_lamb = 0.00415, 0.07686, 0.00768, 0.4585
i_lamb, j_lamb, k_lamb, l_lamb = 0.006579, 0.3856, 0.0217, 1.7083

eta_inft = np.exp(-3.7188 + (578.919 / (-137.546 + T_well)))
T_C = T_well - 273

log_term = (np.log(nC + (cc_eta * T_C + d_eta) - 1) - np.log(e_eta * T_C + f_eta))
factor = (-1 / (a_eta * T_C + b_eta))

eta_0T = eta_inft * (1 + factor * log_term) \
    + (g_eta * T_C + h_eta) * (factor * log_term)**(i_eta * T_C + j_eta) \
    + (k_eta * T_C + l_eta) * (factor * log_term)**(m_eta * T_C + n_eta)

lambda_dC_T = 0.251 + 1.54 * molecular_weight * ceta_exp / (concentration * T_well)

log_term_lamb = (np.log(nC + (g_lamb * T_C + h_lamb) - 1) - np.log(i_lamb * T_C + j_lamb))
factor_lamb = (-1 / (e_lamb * T_C + f_lamb))
lC_T = lambda_dC_T * ((a_lamb * T_C + b_lamb) + (c_lamb * T_C + d_lamb) * factor_lamb * log_term_lamb) ** (k_lamb * T_C + l_lamb)

etaC_T = eta_inft + (eta_0T - eta_inft) * (1 + (lC_T * shear)**2)**((nC - 1) / 2)

# === Final DataFrame ===
df = pd.DataFrame({
    "Shear Rate (1/s)": shear,
    "Viscosity @298K (cP)": etaC,
    f"Viscosity @{T_well_C}°C (cP)": etaC_T
})

print("\nFirst 10 rows of Carreau-Yasuda model at both temperatures:")
print(df.head(10))

# === Plot ===
fig = go.Figure()
fig.add_trace(go.Scatter(x=shear, y=etaC, mode='lines', name='T = 25°C (298 K)'))
fig.add_trace(go.Scatter(x=shear, y=etaC_T, mode='lines', name=f'T = {T_well_C}°C'))
fig.update_layout(
    title="Carreau-Yasuda Viscosity Curve",
    xaxis=dict(title='Shear Rate (1/s)', type='log'),
    yaxis=dict(title='Viscosity η (cP)', type='log'),
    template='plotly_white'
)
fig.show()

# === Save to TXT File ===
df.to_csv("carreau_data_Temp.txt", sep="\t", index=False)
print("Saved data to 'carreau_data_Temp.txt'")



First 10 rows of Carreau-Yasuda model at both temperatures:
   Shear Rate (1/s)  Viscosity @298K (cP)  Viscosity @40°C (cP)
0              0.01             29.701141             19.776739
1              0.11             29.682776             19.770543
2              0.21             29.634287             19.754103
3              0.31             29.556952             19.727643
4              0.41             29.452732             19.691515
5              0.51             29.324119             19.646188
6              0.61             29.173969             19.592226
7              0.71             29.005324             19.530269
8              0.81             28.821258             19.461010
9              0.91             28.624746             19.385179


Saved data to 'carreau_data_Temp.txt'
